In [1]:
%pip install requests
%pip install pandas
%pip install meteostat
%pip install geopy
%pip install scikit-learn
%pip install numpy
%pip install tensorflow-macos tensorflow-metal




  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.4.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.4.26-py3-none-any.whl.metadata (2.5 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached urllib3-2.4.0-py3-none-any.whl (128 kB)
Using cached certifi-2025.4.26-py3-none-any.whl (159 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [requests]
Note: you may need to restart the kernel to use updated packages.
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 15.5 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 17.7 MB/s eta 0:00:00a 0:00:01
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.w

In [2]:
import requests
import pandas as pd

dfs = []

for year in [2020, 2021, 2022, 2023, 2024]:
    url = f'https://incidents.fire.ca.gov/umbraco/api/IncidentApi/List?inactive=true&year={year}'
    print(f'getting data for {year}')
    request = requests.get(url)
    data = request.json()
    df = pd.DataFrame(data)
    df = df[['Started', 'AcresBurned', 'Longitude','Type', 'Latitude', 'ExtinguishedDate']]
    df = df[df['Type'] != 'Earthquake']
    dfs.append(df)

total_frame = pd.concat(dfs, axis=0, ignore_index=True)
print(total_frame.info())
total_frame.to_csv('filtered_incidents.csv', index=False)



getting data for 2020
getting data for 2021
getting data for 2022
getting data for 2023
getting data for 2024
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Started           1338 non-null   object 
 1   AcresBurned       1288 non-null   float64
 2   Longitude         1338 non-null   float64
 3   Type              1338 non-null   object 
 4   Latitude          1338 non-null   float64
 5   ExtinguishedDate  1338 non-null   object 
dtypes: float64(3), object(3)
memory usage: 62.8+ KB
None


In [3]:
from meteostat import Point, Daily
from datetime import datetime, timedelta

row = total_frame.loc[3]

# Parse 'Started' string to datetime
start_date = datetime.strptime(row['Started'], "%Y-%m-%dT%H:%M:%SZ")
end_date = start_date + timedelta(days=1)

# Create location point
loc = Point(row['Latitude'], row['Longitude'])

# Fetch daily weather data
data = Daily(loc, start_date, end_date)
data = data.fetch()

print(data)


            tavg  tmin  tmax  prcp  snow   wdir  wspd  wpgt  pres  tsun
time                                                                   
2020-02-24   4.9  -3.0  14.0   NaN   NaN  342.0   6.7   NaN   NaN   NaN


In [4]:
from meteostat import Stations

stations = Stations()
cali_stations_list = stations.region('US', 'CA')
print(cali_stations_list.count())
cali_stations = cali_stations_list.fetch()

cali_stations.info()




163
<class 'pandas.core.frame.DataFrame'>
Index: 163 entries, 69002 to ZGK9P
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   name           163 non-null    object        
 1   country        163 non-null    string        
 2   region         163 non-null    string        
 3   wmo            53 non-null     string        
 4   icao           159 non-null    string        
 5   latitude       163 non-null    float64       
 6   longitude      163 non-null    float64       
 7   elevation      163 non-null    float64       
 8   timezone       163 non-null    string        
 9   hourly_start   159 non-null    datetime64[ns]
 10  hourly_end     159 non-null    datetime64[ns]
 11  daily_start    154 non-null    datetime64[ns]
 12  daily_end      154 non-null    datetime64[ns]
 13  monthly_start  138 non-null    datetime64[ns]
 14  monthly_end    138 non-null    datetime64[ns]
dtypes: datetime64[ns](

In [5]:
from geopy.distance import geodesic
from collections import defaultdict

# Initialize the storage dictionary
station_fires = defaultdict(list)

# Convert station list to records (so you can easily iterate)
stations = cali_stations.to_dict('records')

# Loop through each fire
for index, fire_row in total_frame.iterrows():
    try:
        fire_point = (fire_row['Latitude'], fire_row['Longitude'])

        # Find closest station
        closest_station = min(
            stations,
            key=lambda s: geodesic(fire_point, (s['latitude'], s['longitude'])).km
        )


        station_id = closest_station['name']  # adjust key name if needed

        # Convert fire row to dict and add to the list for this station
        station_fires[station_id].append(fire_row.to_dict())
    
    except Exception as e:
        print(f"Error at fire index {index}", e)
        continue

# Example: print counts per station
total_fires = 0 
for station_id, fires in station_fires.items():
    total_fires += len(fires)



Error at fire index 167 Latitude must be in the [-90; 90] range.


In [6]:
from datetime import datetime, timedelta

station_daily_status = {}

start_day = datetime(2020, 1, 1)
end_day = datetime.today()
date_range = [start_day + timedelta(days=i) for i in range((end_day - start_day).days - 1)]

for station_id, fires in station_fires.items():

    start_dates = [datetime.strptime(f['Started'], "%Y-%m-%dT%H:%M:%SZ") for f in fires]
    start_dates.sort()

    daily_status = {}
    for current_date in date_range:
        window_start = current_date - timedelta(days=5)
        in_window = any(window_start <= d <= current_date for d in start_dates)
        daily_status[current_date.strftime("%Y-%m-%d")] = in_window

    station_daily_status[station_id] = daily_status

    


In [7]:
from meteostat import Point, Daily
from datetime import datetime

for station_id, daily_status in station_daily_status.items():
    print(f"Processing weather for station {station_id}")
    
    # Get station info
    station_row = cali_stations[cali_stations['name'] == station_id].iloc[0]
    lat = station_row['latitude']
    lon = station_row['longitude']
    
    # Fetch weather data
    point = Point(lat, lon)
    weather = Daily(point, datetime(2020, 1, 1), datetime.today()).fetch()
    if weather.empty:
        print(f"skipping {station_id}")
        continue
    
    # Convert index to string dates
    weather.index = weather.index.strftime("%Y-%m-%d")
    
    for date_str in daily_status:
    # Always unwrap fire flag safely
        fire_flag = daily_status[date_str]
        if isinstance(fire_flag, dict):
            while isinstance(fire_flag, dict):
                fire_flag = fire_flag.get('fire_recent')
        else:
            fire_flag = bool(fire_flag)  # ensure it's boolean

        if date_str in weather.index:
            row = weather.loc[date_str]
            daily_status[date_str] = {
                'fire_recent': fire_flag,
                'tavg': row['tavg'],
                'tmin': row['tmin'],
                'tmax': row['tmax'],
                'prcp': row['prcp'],
                'snow': row['snow'],
                'wdir': row['wdir'],
                'wspd': row['wspd'],
                'wpgt': row['wpgt'],
                'pres': row['pres'],
                'tsun': row['tsun']
            }
        else:
            daily_status[date_str] = {
                'fire_recent': fire_flag,
                'tavg': None,
                'tmin': None,
                'tmax': None,
                'prcp': None,
                'snow': None,
                'wdir': None,
                'wspd': None,
                'wpgt': None,
                'pres': None,
                'tsun': None
            }



Processing weather for station Fortuna / Alton
Processing weather for station Truckee / Polaris
Processing weather for station Weaverville
Processing weather for station Susanville / Johnstonville
Processing weather for station Red Bluff Municipal Airport
Processing weather for station Riverside / Arlanza
Processing weather for station Grass Valley / Forest Knolls
Processing weather for station Murrieta/Temecula / Murrieta Hot Springs (Historical)
Processing weather for station Imperial
Processing weather for station Byron
Processing weather for station Lemoore Naval Air Station
Processing weather for station Santa Ynez
Processing weather for station Beale Afb / Oroville Junction
Processing weather for station Hemet / Egan
Processing weather for station Madera / Notarb
Processing weather for station Santa Rosa / Shiloh
Processing weather for station San Luis Obispo / Steele (Historical)
Processing weather for station Riverside / March Air Force Base
Processing weather for station Lemoo

In [8]:

sample_stations = list(station_daily_status.keys())

for sample_station in sample_stations:
    # Collect dates to remove
    dates_to_remove = []
    
    for date, data in station_daily_status[sample_station].items():
        if isinstance(data, bool):
            dates_to_remove.append(date)
    
    # Remove them after looping (avoid modifying dict during iteration)
    for date in dates_to_remove:
        del station_daily_status[sample_station][date]
    
    print(f"Removed {len(dates_to_remove)} bad entries from station {sample_station}")


Removed 0 bad entries from station Fortuna / Alton
Removed 0 bad entries from station Truckee / Polaris
Removed 0 bad entries from station Weaverville
Removed 0 bad entries from station Susanville / Johnstonville
Removed 0 bad entries from station Red Bluff Municipal Airport
Removed 0 bad entries from station Riverside / Arlanza
Removed 0 bad entries from station Grass Valley / Forest Knolls
Removed 0 bad entries from station Murrieta/Temecula / Murrieta Hot Springs (Historical)
Removed 0 bad entries from station Imperial
Removed 0 bad entries from station Byron
Removed 0 bad entries from station Lemoore Naval Air Station
Removed 0 bad entries from station Santa Ynez
Removed 0 bad entries from station Beale Afb / Oroville Junction
Removed 0 bad entries from station Hemet / Egan
Removed 0 bad entries from station Madera / Notarb
Removed 0 bad entries from station Santa Rosa / Shiloh
Removed 0 bad entries from station San Luis Obispo / Steele (Historical)
Removed 0 bad entries from stati

In [9]:
import pandas as pd

# Flatten to rows

def unwrap_fire_recent(value):
    # Unwrap repeatedly if it's still a dict
    while isinstance(value, dict):
        value = value.get('fire_recent')
    return int(bool(value))  # ensure we get 0 or 1

rows = []

missed_data_count = 0

for station_id, daily_data in station_daily_status.items():
    for date, data in daily_data.items():
        row = {
            'station_id': station_id,
            'date': date,
            'fire_recent': int(data['fire_recent']),  # convert to 0/1
            'tavg': data['tavg'],
            'tmin': data['tmin'],
            'tmax': data['tmax'],
            'prcp': data['prcp'],
            'snow': data['snow'],
            'wdir': data['wdir'],
            'wspd': data['wspd'],
            'wpgt': data['wpgt'],
            'pres': data['pres'],
            'tsun': data['tsun']
        }
        rows.append(row)


df = pd.DataFrame(rows)

df.info()
df.fillna(0, inplace=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253890 entries, 0 to 253889
Data columns (total 13 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   station_id   253890 non-null  object 
 1   date         253890 non-null  object 
 2   fire_recent  253890 non-null  int64  
 3   tavg         247461 non-null  float64
 4   tmin         248772 non-null  float64
 5   tmax         248773 non-null  float64
 6   prcp         239936 non-null  float64
 7   snow         26332 non-null   float64
 8   wdir         242918 non-null  float64
 9   wspd         246385 non-null  float64
 10  wpgt         0 non-null       float64
 11  pres         240616 non-null  float64
 12  tsun         0 non-null       float64
dtypes: float64(10), int64(1), object(2)
memory usage: 25.2+ MB


In [10]:
import pandas as pd

# Assuming you already have df built
fire_df = df[df['fire_recent'] == 1]
nonfire_df = df[df['fire_recent'] == 0]
print(f"Fire days: {len(fire_df)}, Non-fire days: {len(nonfire_df)}")

nonfire_sampled = nonfire_df.sample(n=len(fire_df), random_state=42)
balanced_df = pd.concat([fire_df, nonfire_sampled]).sample(frac=1, random_state=42)  # shuffle

Fire days: 5798, Non-fire days: 248092


In [11]:


feature_cols = ['tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'wpgt', 'pres', 'tsun']
X = balanced_df[feature_cols].values


y = balanced_df['fire_recent'].values





In [12]:

feature_cols = ['tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'wpgt', 'pres', 'tsun']
X = balanced_df[feature_cols].values
y = balanced_df['fire_recent'].values


In [13]:

from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Build Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Evaluate
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")
print(classification_report(y_test, y_pred))

Test Accuracy: 0.7241
              precision    recall  f1-score   support

           0       0.75      0.67      0.71      1160
           1       0.70      0.78      0.74      1160

    accuracy                           0.72      2320
   macro avg       0.73      0.72      0.72      2320
weighted avg       0.73      0.72      0.72      2320



In [14]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred, target_names=['No Fire', 'Fire']))



              precision    recall  f1-score   support

     No Fire       0.75      0.67      0.71      1160
        Fire       0.70      0.78      0.74      1160

    accuracy                           0.72      2320
   macro avg       0.73      0.72      0.72      2320
weighted avg       0.73      0.72      0.72      2320



In [25]:


from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])




In [26]:
optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
290/290 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.5213 - loss: 6.3312 - val_accuracy: 0.6405 - val_loss: 0.6593
Epoch 2/10
290/290 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.5367 - loss: 1.9145 - val_accuracy: 0.5112 - val_loss: 2.0006
Epoch 3/10
290/290 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.5693 - loss: 1.3530 - val_accuracy: 0.5095 - val_loss: 1.5651
Epoch 4/10
290/290 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.5630 - loss: 1.4054 - val_accuracy: 0.5134 - val_loss: 2.0481
Epoch 5/10
290/290 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.5587 - loss: 1.3751 - val_accuracy: 0.6388 - val_loss: 1.0170
Epoch 6/10
290/290 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.5764 - loss: 1.3159 - val_accuracy: 0.5629 - val_loss: 1.7232
Epoch 7/10
290/290 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.5947 - loss: 1.6897 - val_accuracy: 0.4940 - val_loss: 3.7120
Epoch 8/10
290/290 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.5833 - loss: 1.4901 - val_accuracy